In [34]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from copy import deepcopy as dp

In [2]:
from backtest import *

In [3]:
def run_backtest(path_to_data, tickers, strategy, start_date, end_date, commission, 
                strategy_params,
                 viz=True, logging=True):
    '''
    Тестирует стратегию на считанных данных
    '''
    data = warmup(tickers, path_to_data, start_date, end_date, selected_cols)

    strategy.portfolio_init(**strategy_params)

    current_tickers = {}
    pnl = []
    for index, (time, state) in enumerate(data.items()):
        
        des = strategy.get_decision(index, time, state)  # основываясь на текущем портфеле и новой информации, стратегия принимает решение

    return

In [4]:
path_to_data = 'data'
tickers = ['AMD', 'INTC']
start_date = '2015-01-01'
end_date = '2020-01-01'

selected_cols = ['EPS (Basic)']

In [5]:
data_dict = warmup(tickers, path_to_data, start_date, end_date, selected_cols)

AMD (1258, 2)
INTC (1258, 2)


In [56]:
class Strategy():
    '''
    Простая стратегия, которая равномерно покупает компании из всего получаемого списка 
    '''

    def portfolio_init(self, start_money, regular_money, money_freq, decision_freq):
        
        if money_freq < 1:
            print('money_freq should be >= 1!')
            raise
        if decision_freq < 1:
            print('decision_freq should be >= 1!')
            raise
        
        self.regular_money = regular_money
        self.regular_money_freq = money_freq  # количество дней для регулярных поступлений 
        self.portfolio = {'cash': start_money}  # текущее состояние портфеля
        self.days_left = self.regular_money_freq
        self.decision_freq = decision_freq
        self.days_to_decision = self.decision_freq


    def init_ticker(self, ticker):
        self.portfolio[ticker] = {'price': 0, 'volume': 0}
    
    def portfolio_update(self, ticker, new_price, new_amount, add):
        add = True if add == 'to_buy' else False
        if ticker not in self.portfolio:
            self.init_ticker(ticker)
        if ticker == 'cash':
            self.portfolio[ticker] += new_amount
        if add:  # добавляем объем в портфель, перевзвешиваем цену
            cash_spend = new_price * new_amount
            self.portfolio[ticker]['price'] = ((self.portfolio[ticker]['price'] * self.portfolio[ticker]['volume'] + cash_spend) / \
                                               (self.portfolio[ticker]['volume'] + new_amount))
            self.portfolio[ticker]['volume'] += new_amount
            self.portfolio['cash'] -= cash_spend
            if self.portfolio['cash'] < 0:
                print('Negative cash!')
                print(self.portfolio['cash'])
                print(ticker, cash_spend)
                raise
        else:  # продаем, ср. цена та же
            cash_earned = new_price * new_amount
            self.portfolio[ticker]['volume'] -= new_amount
            self.portfolio['cash'] += cash_earned


    def init_decision(self):
        des = {'to_buy': {},
               'to_sell': {}}
        return des 


    def get_decision(self, index, time, state):
        '''
        Сама стратегия. Её можно изменять как угодно
        ''' 
        # FIXME: мб проблема множественного вызова внутри дня

        current_decision = self.init_decision()
        self.days_left -= 1  
        self.days_to_decision -= 1
        
        if self.days_left == 0:
            self.portfolio['cash'] += self.regular_money
            self.days_left = self.regular_money_freq

        # покупаем компании, если возможно
        # докупаю в день денег
        if self.days_to_decision == 0:
            current_money = self.portfolio['cash']
            company_share = current_money / len(state)
            for company, company_state in state.items():  # пробегаем по всем компаниям и записываем решение
                current_decision['to_buy'][company] = company_share  # сколько денег на одну компанию
            self.days_to_decision = self.decision_freq
            
        return current_decision
 

In [57]:
for index, (time, state) in enumerate(data_dict.items()):
    print(time)
    break

2015-01-02


In [58]:
state

{'AMD': {'adj_close': 2.6700000762939453, 'eps_basic': nan},
 'INTC': {'adj_close': 30.671871185302734, 'eps_basic': nan}}

In [90]:
strategy_params = {}
strategy_params['start_money'] = 10000
strategy_params['regular_money'] = 1000
strategy_params['money_freq'] = 1
strategy_params['decision_freq'] = 1

In [91]:
strategy = Strategy()

In [92]:
strategy.portfolio_init(**strategy_params)

In [93]:
old_portfolio = dp(strategy.portfolio)

In [94]:
des = strategy.get_decision(index, time, state)

In [95]:
des

{'to_buy': {'AMD': 5500.0, 'INTC': 5500.0}, 'to_sell': {}}

In [96]:
state

{'AMD': {'adj_close': 2.6700000762939453, 'eps_basic': nan},
 'INTC': {'adj_close': 30.671871185302734, 'eps_basic': nan}}

In [97]:
cash_left = 0
for side, tickers in des.items():
    for ticker, money in tickers.items():
        price = state[ticker]['adj_close']
        amount = money // price
        cashback = money % price
        strategy.portfolio_update(ticker, price, amount, side)
        cash_left += cashback

strategy.portfolio_update(ticker, 1, cash_left, 'cash')

In [98]:
strategy.portfolio

{'cash': 24.409801483154297,
 'AMD': {'price': 2.6700000762939453, 'volume': 2059.0},
 'INTC': {'price': 30.671871185302734, 'volume': 166.79509925842285}}

In [99]:
old_portfolio

{'cash': 10000}